# Question 5 - Visualisations avancées #

In [1]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from visualizations import *
import plotly.graph_objects as go

## 1. Travailler correctement avec les coordonnées de l'événement ##

###  1.1 S'assurer que les tirs sont du bon côté de la patinoire ###
_(en raison de changements de période ou de commencer de différents côtés pendant un match)_

* On s'assure déjà de prendre la distance correcte de chaque tir avec les fonctions '*divide_N_zone*' et '*get_event_distance*' (voir la logique dans le fichier '*visualizations.py*').

* Si ce qu'on veut c'est vraiment savoir de quel côté de la patinoire chaque tir a été effectué, on peut juste regarder si la coordonée X du tir est positive ou négative:
    * **Si x>0:** le tir a été fait depuis le côté droit de la patinoire
    * **Si x<0:** le tir a été fait depuis le côté gauche de la patinoire

### 1.2 Mapper des coordonnées physiques aux coordonnées pixels du graphique ###
* **GAUCHE:** Zone défensive
* **DROITE:** Zone offensive

In [2]:
"""
Cette fonction prend en entrée un évènement, regarde ses coordonnées x et y 
(-100 <= x <= 100 ; -42.4 <= y <= 42.4) et retourne ses coordonnées en pixels
sur le graphique donné (1100 x 467).
"""
def get_pixel_coord(event):
    
    # Dimensions de la patinoire
    field_length = 200
    field_height = 85
    
    # Dimensions de l'image
    img_length = 1100
    img_height = 467
    
    # Récupération des coordonnées
    x_coord = event['X']
    y_coord = event['Y']
    zone = event['Zone']
        
    # Inversion pour les événements offensifs du côté gauche (on les amène au côté droit)
    if zone in ['O', 'NO'] and x_coord < 0:
        x_coord = -x_coord
    
    # --- Transformation en pixels ---
    
    # 1) (coord + field/2) : Translation pour que l'origine (0,0) du terrain (située
    #                        au centre du terrain) se déplace au coin inférieur gauche.
    #    
    #                        On passe de [-L/2, +L/2] à [0, L] pour les coordonnées.
    #
    #                        On réalisera une réflexion par rapport à l'axe X lors de 
    #                        l’affichage du graphique pour inverser les valeurs dans Y et
    #                        passer l=origine au coin supérieur gauche,car dans une image
    #                        l’axe Y augmente vers le bas.
    #
    # 2) / field : Normalisation pour obtenir des valeurs entre 0 et 1.
    #
    # 3) * img : Mise à l’échelle en pixels.
    
    x_pix = (x_coord + field_length / 2) / field_length * img_length
    y_pix = (y_coord + field_height/2) / field_height * img_height

    return round(x_pix), round(y_pix)



"""
Cette fonction ajoute les coordonnées en pixels à chaque évènement dans notre DataFrame de saison.
"""
def add_events_pixel_coord(df):
    
    # Éliminer les évènements avec coordonnées manquantes
    df = df.dropna(subset=['X', 'Y']).copy()
    
    # Trouver les coordonnées en pixels pour chaque évènement
    df[['Pixel X', 'Pixel Y']] = df.apply(
        get_pixel_coord,
        axis=1,                 # Appliquer sur les lignes
        result_type='expand'    # Diviser le resultat en 2 colonnes
    )
    
    return df

## 2. Taux de tir moyen par heure de la ligue par emplacement ##
_(On suppose que chaque match dure 60 minutes)_ 

### On divisera le terrain en cases (bins) pour calculer le taux de tirs sur chaque zone au lieu de le calculer sur chaque position [x_pix, y_pix] individuelle. ###

In [3]:
"""
Cette fonction prend en arguments des coordonées x et y (en pixels) et retourne les 
coordonnées du centre de la case/bin à laquelle appartient le point (x, y).
"""
def bin_coordinates(x, y, bin_size_x, bin_size_y):
    
#     1) (coord // bin_size) : Détermine l'indice du bin auquel le pixel appartient.
#
#     2) * bin_size : Calcule la coordonnée de départ du bin. 
#
#     3) + bin_size // 2 : Ajoute la moitié de la taille du bin pour déplacer la coordonnée au centre.          
    
    bin_x = (x // bin_size_x) * bin_size_x + bin_size_x // 2 
    bin_y = (y // bin_size_y) * bin_size_y + bin_size_y // 2 
    
    return bin_x, bin_y



"""
Cette fonction prend un DataFrame de fréquence par coordonnées, regroupe ses coordonnées
par bins d'une taille donnée et calcule la somme des fréquences pour chaque bin.
"""
def group_by_bins(frequency_df, bin_size_x, bin_size_y):
    
    # Ajouter les coordonnées du centre du bin auquel chaque coordonnée d'évènements appartient
    frequency_df['Bin X'], frequency_df['Bin Y'] = bin_coordinates(
        frequency_df['Pixel X'].values,
        frequency_df['Pixel Y'].values,
        bin_size_x,
        bin_size_y
    )
    
    # Agrouper les coordonnées d'évènements par bins et faire la somme des fréquences pour chaque bin
    binned_df = frequency_df.groupby(['Bin X', 'Bin Y'], as_index=False)['Frequency'].sum()
    
    return binned_df

### 2.1 Calcul des statistiques agrégées des emplacements de tir dans l'ensemble de la ligue ###

In [4]:
"""
Cette fonction prend en entrée un DataFrame de saison (avec les coordonnées en pixels)
et calcule la fréquence des emplacements (en pixels) sur tous les tirs de la ligue.
"""
def shots_location_frequency(df):
    
    # Compter le nombre d'occurences de chaque paire (x, y)
    coords_frequency_group = df.groupby(["Pixel X", "Pixel Y"]).size()
    
    # Tranformer en DataFrame
    coords_frequency_df = coords_frequency_group.reset_index(name='Frequency')
    
    return coords_frequency_df

### 2.2 Calcul du taux de tir moyen par heure de la ligue par emplacement ###

In [5]:
"""
Cette fonction prend en entrée un DataFrame de saison (avec les coordonnées en pixels)
et calcule le taux de tir moyen par heure de la ligue pour chaque bin.
"""
def get_avg_hourly_shot_rate_per_bin(df, bin_size_x, bin_size_y):
    
    # Nombre total de matchs
    games_nb = df['Game ID'].nunique()

    # Nombre total de tirs par emplacement
    location_frequency_df = shots_location_frequency(df)
    
    # Nombre total de tirs par bin
    binned_df = group_by_bins(location_frequency_df, bin_size_x, bin_size_y)
    
    # Calculer le taux de tir moyen par heure par bin
    binned_df['Hourly Rate'] = binned_df['Frequency'] / (games_nb * 2)  # * 2 car il y a 2 équipes par game
    
    # Éliminer la colonne des fréquences
    league_avg_rate_df = binned_df[['Bin X', 'Bin Y', 'Hourly Rate']]
    
    return league_avg_rate_df


## 3. Différence du taux de tir par heure pour chaque équipe avec la moyenne ##
_(Différence représentée en différence brute)_

### 3.1 Regroupement des tirs par équipe ###

In [6]:
"""
Cette fonction prend en entrée un DataFrame de saison (avec les coordonnées en pixels) et 
calcule la fréquence par équipe des emplacements (en pixels) sur tous les tirs de la ligue.
"""
def teams_shot_location_frequency(df):
    
    # Compter le nombre d'occurences de chaque paire (x, y)
    coords_frequency_group = df.groupby(["Team", "Pixel X", "Pixel Y"]).size()
    
    # Tranformer en DataFrame
    coords_frequency_df = coords_frequency_group.reset_index(name='Frequency')
    
    return coords_frequency_df

### 3.2 Calcul de la différence du taux de tir par heure pour chaque équipe avec la moyenne ###

In [7]:
"""
Cette fonction prend un DataFrame de fréquence par coordonnées de toutes les équipes
et calcule, pour une équipe donnée, le taux horaire de tirs par bin.
"""
def get_team_hourly_shot_rate_per_bin(teams_frequency_df, team, games_nb, bin_size_x, bin_size_y):

    # Nombre total de tirs de l'équipe par emplacement
    team_frequency_df = teams_frequency_df[teams_frequency_df["Team"] == team].copy()
    
    # Nombre total de tirs par bin
    binned_df = group_by_bins(team_frequency_df, bin_size_x, bin_size_y)
    
    # Taux par heure
    binned_df['Hourly Rate'] = binned_df['Frequency'] / games_nb
    
    # Ajouter le nom de l'equipe en colonne
    binned_df['Team'] = team
    
    # Éliminer la collone des fréquences
    team_rate_df = binned_df[['Team', 'Bin X', 'Bin Y', 'Hourly Rate']]
    
    return team_rate_df


"""
Cette fonction prend en entrée un DataFrame de saison (avec les coordonnées en pixels) 
et calcule, pour chaque équipe de la ligue, le taux de tirs par heure dans chaque bin.
"""
def get_teams_hourly_shot_rate(df, bin_size_x, bin_size_y):
    
    # Liste d'équipes dans la ligue
    teams = df['Team'].unique().tolist()
    
    # Dictionnaire avec ombre total de matchs joués par chaque équipe 
    games_per_team = df.groupby('Team')['Game ID'].nunique().to_dict()
    
    # Nombre total de tirs par emplacement pour chaque équipe
    teams_frequency_df = teams_shot_location_frequency(df)
    
    # Liste avec les taux de tirs par heure et par bin de chaque équipe
    teams_rates_dfs = []
    
    # Pour chaque équipe de la lige
    for team in teams:
        
        # Nombre de matchs joués par cette équipe
        games_nb = games_per_team[team]
        
        # Taux de tirs par heure et par bin pour cette équipe
        team_rate_df = get_team_hourly_shot_rate_per_bin(teams_frequency_df, team, games_nb, bin_size_x, bin_size_y)
        
        # Ajouter à la liste
        teams_rates_dfs.append(team_rate_df)
        
    # Unifier en un seul DataFrame
    teams_rates_df = pd.concat(teams_rates_dfs, ignore_index=True)
    
    return teams_rates_df

In [8]:
"""
Cette fonction prend en entrée un DataFrame de saison (avec les coordonnées en pixels) et calcule 
la différence du taux de tir par heure par bin de chaque équipe avec celui de la ligue.
"""
def get_rate_diff_teams_vs_league(df, bin_size_x, bin_size_y):
    
    # Taux moyen ligue par bin
    league_avg_rate_df = get_avg_hourly_shot_rate_per_bin(df, bin_size_x, bin_size_y)
    league_avg_rate_df = league_avg_rate_df.rename(columns={'Hourly Rate': 'League Hourly Rate'})
    
    # Taux par équipe par bin
    team_rate_df = get_teams_hourly_shot_rate(df,  bin_size_x, bin_size_y)
    
    # Fusionner sur Bin X, Bin Y
    merged_df = team_rate_df.merge(league_avg_rate_df, on=['Bin X', 'Bin Y'], how='left')
    
    # Différence brute
    merged_df['Difference'] = merged_df['Hourly Rate'] - merged_df['League Hourly Rate']
    
    return merged_df

## 4. Regrouper les données ##
_(Lissage réalisé avec une estimation de densité de noyau gaussien)_ 

In [9]:
"""
Cette fonction crée une grille lissée des différences de taux pour une équipe donnée, 
en remplissant chaque pixel du bin avec sa valeur.
"""
def get_team_diff_grid(rate_diff_df, team_name, bin_size_x, bin_size_y, sigma):
    
    # Dimensions de l'image
    img_x = 1100
    img_y = 467
    
    # Initialisation de la grille
    grid = np.full((img_y, img_x), 0.0)
    
    # Différences de taux pour cette équipe
    team_df = rate_diff_df[rate_diff_df['Team'] == team_name][['Bin X', 'Bin Y', 'Difference']].copy()

    
    # Pour chaque bin
    for _, row in team_df.iterrows():
        
        # Coordonnées du bin
        bin_center_x = int(row['Bin X'])
        bin_center_y = int(row['Bin Y'])
        
        # Différence de taux pour ce bin
        diff_value = float(row['Difference'])
        
        # Limites du bin
        x_start = bin_center_x - bin_size_x // 2
        x_end = bin_center_x + bin_size_x // 2
        y_start = bin_center_y - bin_size_y // 2
        y_end = bin_center_y + bin_size_y // 2
        
        # S'assurer que les limites restent dans les dimensions de la grille
        x_start = max(0, x_start)
        x_end = min(img_x, x_end)
        y_start = max(0, y_start)
        y_end = min(img_y, y_end)
        
        # Remplir tous les pixels du bin avec la valeur de différence
        grid[y_start:y_end, x_start:x_end] = diff_value
    
    # Lissage de la grille
    smooth_grid = gaussian_filter(grid, sigma=sigma)
    
    # Supprimer les valeurs situées aux bords de la grille 
    # (pour des raisons purement esthétiques)
    smooth_grid[:, :690] = 0
    smooth_grid[:, 1039:] = 0
    smooth_grid[:2, :] = 0
    smooth_grid[465:, :] = 0
    
    return np.around(smooth_grid, 3)
    



"""
Cette fonction prend en entrée un DataFrame de saison et et retourne un dictionnaire 
contenant, pour chaque équipe, une grille lissée des différences de taux de tirs par bin.
"""
def get_teams_grid(season_df, bin_size_x = 55, bin_size_y = 55, sigma = 12):
    
    # Garder juste les tirs en zone offensive
    offensive_season_df = season_df[(season_df['Zone'] == 'O')]
    
    # Calculer les coordonnées en pixels pour chaque évènement
    pixel_coords_season_df = add_events_pixel_coord(offensive_season_df)
    
    # DataFrame avec les différences de taux pour chaque équipe
    rate_diff_df = get_rate_diff_teams_vs_league(pixel_coords_season_df, bin_size_x, bin_size_y)

    # Équipes de la ligue
    teams = rate_diff_df['Team'].unique().tolist()

    # Clé: nom de l'équipe ; Valeur: Grille avec les différences de taux
    grids_dict = {}

    # Faire une grille pour chaque equipe
    for team in teams:
        grid = get_team_diff_grid(rate_diff_df, team, bin_size_x, bin_size_y, sigma=sigma)
        grids_dict[team] = grid
    
    return grids_dict

## 5.  Graphique interactif ##

In [10]:
"""
Cette fonction crée une échelle de couleurs discrète à partir d'une liste de couleurs, 
chaque couleur occupant une portion égale de l'échelle.
"""
def make_discrete_colorscale(colors):
    
    # Nombre de couleurs
    n = len(colors)

    # Taille de la portion de l'échelle pour chaque couleur 
    step = 1 / n
    
    # Échelle de couleurs
    colorscale = []

    # Pour chaque couleur, définir sa position sur l'échelle
    for i, color in enumerate(colors):
        start = i * step       # Début
        end = (i + 1) * step   # Fin

        colorscale.append([start, color])
        colorscale.append([end, color])

    return colorscale

In [11]:
"""
Cette fonction crée un heatmap dynamique des différences de taux de tirs par heure
pour toutes les équipes d'une saison. Possibilité d'exporter le graphique en HTML.
"""
def plot_dynamic_teams_heatmap(season, season_df, bin_size_x, bin_size_y, sigma, export=False):
    
    
    # ---- PRÉPARATION DES DONNÉES ----
    
    # Calculer les coordonnées en pixels pour chaque évènement
    season_df = add_events_pixel_coord(season_df)
    
    # Grilles lissées des différences de taux de chaque équipe
    grids_dict = get_teams_grid(season_df, bin_size_x, bin_size_y, sigma=sigma)
    
    # Liste des équipes
    teams = sorted(list(grids_dict.keys()))

    # Grille initiale (équipe au chargement)
    initial_team = teams[0]
    z_initial = grids_dict[initial_team]
    
    # Couleurs pour l'échelle
    colors = [
        'rgba(68, 8, 125, 1)',    # Bleue
        'rgba(59, 6, 134, 1)',
        'rgba(45, 3, 158, 1)',
        'rgba(32, 2, 187, 1)',
        'rgba(17, 1, 211, 1)',
        'rgba(0, 0, 234, 1)',
        'rgba(40, 40, 245, 1)',
        'rgba(90, 90, 246, 1)',
        'rgba(145, 145, 248, 1)',
        'rgba(199, 199, 251, 1)',
        'rgba(255, 255, 255, 0)',  # Transparent
        'rgba(247, 201, 200, 1)',
        'rgba(240, 151, 148, 1)',
        'rgba(235, 101, 96, 1)',
        'rgba(234, 66, 55, 1)',
        'rgba(224, 49, 34, 1)',
        'rgba(201, 43, 29, 1)',
        'rgba(173, 35, 23, 1)',
        'rgba(151, 29, 19, 1)',
        'rgba(128, 23, 14, 1)',
        'rgba(113, 19, 11, 1)'     # Rouge
    ]
    
    # Créer l'échelle de couleurs
    colorscale = make_discrete_colorscale(colors)
    
    
    
    
    # ---- CRÉATION DU HEATMAP DYNAMIQUE ----

    fig = go.Figure()

    # Heatmao initial
    fig.add_trace(go.Heatmap(
        z=z_initial,
        hovertemplate="Diff: %{z:.3f}<extra></extra>",
        colorbar=dict(
            tickmode="array",
            tickvals=[i/10 for i in range(-10, 11, 2)],
            ticktext=[
                f"{i/10:.1f}" if i < 0 else f"+{i/10:.1f}"
                for i in range(-10, 11, 2)
            ],
            len=1.1,
            thickness=17,
            xpad=30,
        ),
        colorscale=colorscale,
        zmin=-1,
        zmax=1,
        )
    )

    # Boutons pour chager d'équipe
    buttons = []
    for team in teams:
        z = grids_dict[team]
        button = dict(
            label=team,
            method="update",
            args=[
                {"z": [z]},
            ]
        )
        buttons.append(button)


    # Mise en page
    fig.update_layout(
        width=950,
        height=550,
        plot_bgcolor='white',
        # Menu des équipes
        updatemenus=[
            dict(
                buttons=buttons,
                x=0.505,
                y=1.125,
                direction="down",
                showactive=True,
                xanchor="left",
                yanchor="top",
                pad={"r": 2, "t": 0},
                bgcolor="white",
                bordercolor="gray",
                borderwidth=1
            )
        ],
        # Titre principal
        title=dict(
            text=f"Différence du taux de tir par heure avec la moyenne ({season})",
            x=0.5,
            y=0.96,
            xanchor="center",
            yanchor="top",
            font=dict(size=20),
        ),
        # Axe X
        xaxis_title="X (ft)",
        xaxis=dict(
            range=[0, 1100],
            showgrid=False,
            zeroline=False,
            scaleanchor="y",
            scaleratio=1,
            linecolor='black',
            ticks='outside',
            ticklen=5,
            tickcolor='black',
            tickmode='array',
            tickvals=[i * 5.5 + 550 for i in range(-100, 101, 10)],
            ticktext=[str(i) for i in range(-100, 101, 10)],
        ),
        # Axe Y
        yaxis_title="Y (ft)",
        yaxis=dict(
            autorange="reversed",
            title_standoff=6,
            range=[0,467],
            showgrid=False,
            zeroline=False,
            linecolor='black',
            ticks='outside',
            ticklen=5,
            tickcolor='black',
            tickmode='array',
            tickvals=[(i + 42.5) * (467 / 85) for i in range(40, -41, -10)],
            ticktext=[str(i) for i in range(-40, 41, 10)],
        ),
        # Image de fond
        images=[dict(
            source="../../figures/nhl_rink.png",
            xref="x",
            yref="y",
            x=0,
            y=0,
            sizex=1100,
            sizey=467,
            sizing="stretch",
            opacity=1,
            layer="below"
        )],
    )
    
    
    
    
    # ---- TEXTES ----
    
    # Échelle
    fig.add_annotation(
        text="Différence",
        x=1.125,
        y=1.14,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=14.5, color="#444"),
    )
    fig.add_annotation(
        text="(Tirs)",
        x=1.099,
        y=1.09,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=12, color="#444"),
    )
    fig.add_annotation(
        text=f"Par zone de {bin_size_x}x{bin_size_y}px",
        x=1.073, y=-0.03,
        ax=-30, ay=37, 
        xref="paper",
        yref="paper",
        showarrow=True,
        font=dict(size=11, color="#444"),
    )
    fig.add_annotation(
        text=f"Lissage avec σ={sigma}",
        x=1.105, y=-0.195,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=11, color="#444"),
    )

    # Zones du terrain
    fig.add_annotation(
        text="Zone défensive",
        x=0.125,
        y=0.995,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=14.5, color="gray")
    )
    fig.add_annotation(
        text="Zone neutre",
        x=0.5,
        y=0.995,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=14.5, color="gray")
    )
    fig.add_annotation(
        text="Zone offensive",
        x=0.875,
        y=0.995,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=14.5, color="gray"),
    )

    # Étiquette des boutons
    fig.add_annotation(
        text="Équipe:",
        x=0.455,
        y=1.11,
        xref="paper",
        yref="paper",
        showarrow=False,
        font=dict(size=16, color="black")
    )
    
    # Exporter en HTML 
    if export:
        fig.write_html(f"{season}_dynamic_heatmap.html")
    
    fig.show()

## 6. Graphique interactif pour chaque saison de 2016-2017 à 2020-2021 ##
_Figures uniquement de la zone offensive_

### Saison 2016-2017 ###

In [ ]:
df = get_season_df('2016-2017')
plot_dynamic_teams_heatmap('2016-2017', df, 55, 66, 12, False)

### Saison 2017-2018 ###

In [ ]:
df = get_season_df('2017-2018')
plot_dynamic_teams_heatmap('2017-2018', df, 55, 66, 12, False)

### Saison 2018-2019 ###

In [ ]:
df = get_season_df('2018-2019')
plot_dynamic_teams_heatmap('2018-2019', df, 55, 66, 12, False)

### Saison 2019-2020 ###

In [ ]:
df = get_season_df('2019-2020')
plot_dynamic_teams_heatmap('2019-2020', df, 55, 66, 12, False)

### Saison 2020-2021 ###

In [ ]:
df = get_season_df('2020-2021')
plot_dynamic_teams_heatmap('2020-2021', df, 55, 66, 12, False)